# Esercizio 7

Estrarre da un file in formato `FASTQ` i reads che hanno una percentuale almeno P di basi con qualità almeno Q (P e Q sono parametri in input). Per ognuno di tali reads produrre in output (in formato FASTQ) la più lunga sottostringa in cui tutte le basi hanno qualità almeno Q, ma solo se la sua lunghezza supera una soglia minima L (altrimenti il read deve essere scartato).

***

Parametri in input:

- percentuale minima P (numero compreso tra 0 e 1)
- soglia minima di qualità Q
- lunghezza minima L

***

Requisiti:

- deve essere definita la funzione `ascii_to_quality()` che prende come argomento un carattere ASCII e restituisce il corrispondente Phred Value q.
- deve essere definita la funzione `get_quality_percentage()` che prende in input una stringa di qualità (cioé una stringa di caratteri ASCII che codificano Phred Values) e una soglia minima, e restituisce la percentuale di caratteri che codificano una qualità pari ad almeno la soglia minima.
- deve essere definita la funzione `get_trimming_interval()` che prende in input una stringa di qualità e una soglia minima, e restituisce il più lungo intervallo di posizioni contenente solo caratteri che codificano qualità almeno pari alla soglia minima.
- deve essere definita la funzione `get_trimmed_read()` che prende in input una soglia minima di qualità e un read in  formato `FASTQ` inteso come lista dei suoi quattro *record* `FASTQ`, ed effettua il *trimming*, cioé trova la più lunga sottostringa del read le cui basi hanno una qualità pari ad almeno la soglia passata come argomento. La funzione deve restituire la lista dei quattro *record* `FASTQ` del *trimmed* read.


**Esempio:** per Q=58.

Read prima del *trimming*:

        @HWUSI-EAS522:8:5:662:692#0/1
        TATGGAGGCCCAACTTCTTGTATTCACAGGTTCTGC
        +HWUSI-EAS522:8:5:662:692#0/1
        aaaa`aa`aa`]__`aa`_U[_a`^\\UTWZ`X^QX
        
Read dopo il *trimming*:

        @HWUSI-EAS522:8:5:662:692#0/1
        TATGGAGGCCCAACTTCTT
        +HWUSI-EAS522:8:5:662:692#0/1,
        aaaa`aa`aa`]__`aa`_
        
***

***

## Soluzione

### Parametri in input

In [1]:
fastq_file_name = './input.fq'

Soglia minima di qualità (parametro Q):

In [2]:
quality_threshold = 58

Minima percentuale di basi aventi qualità minima Q (parametro P):

In [3]:
min_percentage = 0.7

Lunghezza minima della parte rimasta dopo il trimming (paramtero L):

In [4]:
min_length = 20

### Definizione della funzione `ascii_to_quality()`

La funzione prende come argomento un carattere `c` e restituisce il valore di qualit `q` da esso codificato secondo la codifica di Illumina:

    q = ASCII(min(93,c)-33)

### Definizione della funzione `get_quality_percentage()`

La funzione prende come argomento una stringa di valori di qualità, codificati in caratteri ASCII, e una soglia minima di qualità e restituisce la percentuale (come valore numero tra 0 e 1) di caratteri che codificando una qualità maggiore o uguale alla soglia minima.

### Definizione della funzione `get_trimming_interval()`

La funzione prende come argomento una stringa di valori di qualità, codificati in caratteri ASCII, e soglia minima di qualità e il più lungo intervallo di posizioni contenente solo caratteri che codificano qualità almeno pari alla soglia minima.

#### Spiegazione della funzione `get_trimming_interval()`

Primo argomento:

In [17]:
quality_string = 'aaaa`aa`aa`]__`aa`_U[_a`^\\\\UTWZ`X^QX'

Secondo argomento:

In [23]:
quality_threshold

58

**Determinare il più lungo intervallo di posizioni che contiene solo indici di qualità che superano la soglia minima**
    
Determinare la lista di tutte le posizioni di inizio di un intervallo massimale contenente indici di qualità pari ad almeno la soglia minima.

Una posizione `i` nella stringa `quality_string` è di inizio di un intervallo massimale di valori di qualità pari ad almeno la soglia minima se l'indice di qualità alla posizione `i` e almeno pari a `quality_threshold` e quello alla posizione `i-1` è al di sotto.

`i=0` è una posizione di inizio di un intervallo massimale se l'indice qualità alla posizione 0 è pari ad almeno la soglia minima.

In [79]:
start_list

[0, 20, 31, 33]

Aggiungere come primo elemento della lista una *fake position* pari a 1, per gestire il caso in cui tutti i valori di qualità sono al di sotto della soglia minima.

In [81]:
start_list

[1, 0, 20, 31, 33]

Costruire in modo analogo la lista di tutte le posizioni di fine di un intervallo massimale.

Una posizione `j` nella stringa `quality_string` è di fine di un intervallo massimale di valori di qualità pari ad almeno la soglia minima se l'indice di qualità alla posizione `j` e almeno pari a `quality_threshold` e quello alla posizione `j+1` è al di sotto.

`j=len(quality_string)` è una posizione di fine di un intervallo massimale se l'indice qualità alla posizione `len(quality_string)` è pari ad almeno la soglia minima.

In [90]:
end_list

[18, 26, 31, 33]

Aggiungere come primo elemento una *fake position* pari a 0, per gestire il caso in cui tutti i valori di qualità sono al di sotto della soglia minima.

In [92]:
end_list

[0, 18, 26, 31, 33]

`start_list[p]` ed `end_list[p]` sono le due posizioni di inizio e di fine sulla stringa di qualità di un intervallo massimale di soli indici di qualità pari ad almeno la soglia minima. Quindi `end_list[p]-start_list[p]+1` è la lunghezza del p-esimo intervallo massimale a partire da sinistra.

Per `p=0` si ha `start_list[0]=-1` e `end_list[0]=-2`, che sono le posizioni di inizio e fine del *fake interval* che avrà lunghezza 0.

A questo punto basta determinare l'intervallo più lungo.

Costruire quindi la lista delle lunghezza degli intervalli massimali trovati.

In [94]:
interval_lengths

[0, 19, 7, 1, 1]

Estrarre l'inizio e la fine dell'intervallo di lunghezza massima (che sarà quindi il *trimming interval*).

In [96]:
interval_start

0

In [97]:
interval_end

18

Si faccia in modo che la funzione `get_trimming_interval()` restituisca la lista dei due elementi `[interval_start, interval_end+1]` che rappresenta il *trimming interval*.

Il fatto di sommare 1 a `interval_end` prima di restituirlo è per rendere l'intervallo subito pronto per effettuare l'operazione di *slicing* (trimming) della stringa di qualità e del read.

**NB**: se viene restituito il *fake interval* `[1,1]` allora significa che tutti i valori in `bool_list` sono uguali a `False` (cioé nella stringa di qualità tutti i valori sono al di sotto della soglia minima), e il trimming restituirà una stringa vuota.

### Definizione della funzione `get_trimmed_read()`

La funzione prende come primo argomento una soglia minima di qualità e come argomento la lista dei quattro *record* di un read in formato `FASTQ` e restituisce la lista dei *record* `FASTQ` del read dopo essere stato sottoposto a trimming.

---

### Lettura del file in formato `FASTQ`

Lettura del file `FASTQ` in input.

In [101]:
input_file_rows

['@HWUSI-EAS522:8:5:662:692#0/1\n',
 'TATGGAGGCCCAACTTCTTGTATTCACAGGTTCTGC\n',
 '+HWUSI-EAS522:8:5:662:692#0/1\n',
 'aaaa`aa`aa`]__`aa`_U[_a`^\\\\UTWZ`X^QX\n',
 '@HWUSI-EAS522:8:5:662:693#0/1\n',
 'TCTGCCAACTTCTTATGGAGGCCTGTATTCACAGGT\n',
 '+HWUSI-EAS522:8:5:662:693#0/1\n',
 'Aaaa`aa`aa`]__`:a`_U;_A`^\\\\UTWZ`X^QX\n',
 '@HWUSI-EAS522:8:5:662:694#0/1\n',
 'TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTATG\n',
 '+HWUSI-EAS522:8:5:662:694#0/1\n',
 'aaaa`aa`aa`]__`aa`_u[_a`^\\\\utwz`x^QX\n',
 '@HWUSI-EAS522:8:5:662:695#0/1\n',
 'TCGCCTGTATTCACAGGTTGCCAACTTCTTATGGAG\n',
 '+HWUSI-EAS522:8:5:662:695#0/1\n',
 'AaaA`aa`aa`]__`:A`_U;_A`^\\\\UTWZ`X^QX\n']

Raggruppare i *record* per read (cioé a gruppi di quattro) e inserirli in una lista di liste, in modo tale che ogni lista interna contenga le quattro stringhe del record FASTQ.

In [103]:
fastq_read_list

[['@HWUSI-EAS522:8:5:662:692#0/1\n',
  'TATGGAGGCCCAACTTCTTGTATTCACAGGTTCTGC\n',
  '+HWUSI-EAS522:8:5:662:692#0/1\n',
  'aaaa`aa`aa`]__`aa`_U[_a`^\\\\UTWZ`X^QX\n'],
 ['@HWUSI-EAS522:8:5:662:693#0/1\n',
  'TCTGCCAACTTCTTATGGAGGCCTGTATTCACAGGT\n',
  '+HWUSI-EAS522:8:5:662:693#0/1\n',
  'Aaaa`aa`aa`]__`:a`_U;_A`^\\\\UTWZ`X^QX\n'],
 ['@HWUSI-EAS522:8:5:662:694#0/1\n',
  'TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTATG\n',
  '+HWUSI-EAS522:8:5:662:694#0/1\n',
  'aaaa`aa`aa`]__`aa`_u[_a`^\\\\utwz`x^QX\n'],
 ['@HWUSI-EAS522:8:5:662:695#0/1\n',
  'TCGCCTGTATTCACAGGTTGCCAACTTCTTATGGAG\n',
  '+HWUSI-EAS522:8:5:662:695#0/1\n',
  'AaaA`aa`aa`]__`:A`_U;_A`^\\\\UTWZ`X^QX\n']]

Eliminare dalla lista i read in cui la percentuale di basi con qualità almeno pari alla soglia minima (parametro `quality_threshold`) è inferiore al parametro `min_percentage`.

In [105]:
fastq_read_list

[['@HWUSI-EAS522:8:5:662:692#0/1\n',
  'TATGGAGGCCCAACTTCTTGTATTCACAGGTTCTGC\n',
  '+HWUSI-EAS522:8:5:662:692#0/1\n',
  'aaaa`aa`aa`]__`aa`_U[_a`^\\\\UTWZ`X^QX\n'],
 ['@HWUSI-EAS522:8:5:662:694#0/1\n',
  'TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTATG\n',
  '+HWUSI-EAS522:8:5:662:694#0/1\n',
  'aaaa`aa`aa`]__`aa`_u[_a`^\\\\utwz`x^QX\n']]

Effettuare il *trimming* dei reads rimasti.

In [107]:
fastq_read_list

[['@HWUSI-EAS522:8:5:662:692#0/1\n',
  'TATGGAGGCCCAACTTCTT\n',
  '+HWUSI-EAS522:8:5:662:692#0/1\n',
  'aaaa`aa`aa`]__`aa`_\n'],
 ['@HWUSI-EAS522:8:5:662:694#0/1\n',
  'TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTA\n',
  '+HWUSI-EAS522:8:5:662:694#0/1\n',
  'aaaa`aa`aa`]__`aa`_u[_a`^\\\\utwz`x^\n']]

Eliminare i reads troppo corti.

In [109]:
fastq_read_list

[['@HWUSI-EAS522:8:5:662:694#0/1\n',
  'TCTGCCAGAGGCCTGTATTCACAGGTACTTCTTA\n',
  '+HWUSI-EAS522:8:5:662:694#0/1\n',
  'aaaa`aa`aa`]__`aa`_u[_a`^\\\\utwz`x^\n']]